<a href="https://colab.research.google.com/github/gogamid/ml-notebooks/blob/main/sccl-experiments/ewc_avalanche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-0qn7j296
  Running command git clone --filter=blob:none --quiet https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-0qn7j296
  Resolved https://github.com/ContinualAI/avalanche.git to commit 39c097fde7953e1b4515efa42503421bcac4ba88
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.evaluation.metrics import accuracy_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import EWC
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

benchmark = PermutedMNIST(10)

model = SimpleMLP(
    num_classes=benchmark.n_classes,
    hidden_size=512,
    hidden_layers=1,
    drop_rate=0
)

loggers=[InteractiveLogger(),TextLogger(open('logPermutedMNIST.txt', 'a'))]
loggers.append(WandBLogger(project_name="avalanche", run_name="ewc_perm_mnist"))


eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loggers=loggers
)

cl_strategy = EWC(
    model,
    SGD(model.parameters(), lr=0.001),
    CrossEntropyLoss(),
    ewc_lambda=1,
    mode="separate",
    decay_factor=None,
    train_mb_size=256,
    train_epochs=10,
    eval_mb_size=128,
    device=device,
    evaluator=eval_plugin
)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print( "Start of experience: ", experience.current_experience)
    print( "Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

wandb.finish()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Starting experiment...
-- >> Start of training phase << --
100%|██████████| 235/235 [00:12<00:00, 19.11it/s]
Epoch 0 ended.
100%|██████████| 235/235 [00:12<00:00, 19.31it/s]
Epoch 1 ended.
100%|██████████| 235/235 [00:12<00:00, 18.17it/s]
Epoch 2 ended.
100%|██████████| 235/235 [00:13<00:00, 17.51it/s]
Epoch 3 ended.
100%|██████████| 235/235 [00:14<00:00, 16.23it/s]
Epoch 4 ended.
100%|██████████| 235/235 [00:12<00:00, 19.25it/s]
Epoch 5 ended.
100%|██████████| 235/235 [00:12<00:00, 18.79it/s]
Epoch 6 ended.
100%|██████████| 235/235 [00:12<00:00, 18.87it/s]
Epoch 7 ended.
100%|██████████| 235/235 [00:12<00:00, 18.14it/s]
Epoch 8 ended.
100%|██████████| 235/235 [00:12<00:00, 18.93it/s]
Epoch 9 ended.
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:02<00:00, 36.88it/s]
> Eval on experience 0 (Task 0) from test stream ended.

Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000,█▇▆▅▅▅▂▁▂▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001,▁███▇▇▇▇▇▇
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002,▁▁██████▇▇
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003,▁▁▁█████▇▇
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004,▁▁▁▁█████▇
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp005,▁▁▂▁▁█████
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp006,▁▁▂▂▁▁███▇
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp007,▁▁▁▁▁▁▁███
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp008,▁▁▁▁▁▁▁▁██
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009,▁▁▂▁▁▁▁▁▁█
Top1_Acc_Stream/eval_phase/test_stream/Task000,▁▂▃▃▄▅▆▆▇█


In [3]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import accuracy_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import EWC

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

benchmark = SplitMNIST(n_experiences=5)

model = SimpleMLP(
    num_classes=benchmark.n_classes,
    hidden_size=512,
    hidden_layers=1,
    drop_rate=0
)

loggers=[InteractiveLogger(),TextLogger(open('logSplitMNIST.txt', 'a'))]
loggers.append(WandBLogger(project_name="avalanche", run_name="ewc_split_mnist"))


eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loggers=loggers
)
cl_strategy = EWC(
    model,
    SGD(model.parameters(), lr=0.001),
    CrossEntropyLoss(),
    ewc_lambda=1,
    mode="separate",
    decay_factor=None,
    train_mb_size=256,
    train_epochs=10,
    eval_mb_size=128,
    device=device,
    evaluator=eval_plugin
)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))


wandb: Currently logged in as: gogamid (sccl). Use `wandb login --relogin` to force relogin


Starting experiment...
-- >> Start of training phase << --
100%|██████████| 47/47 [00:02<00:00, 23.05it/s]
Epoch 0 ended.
100%|██████████| 47/47 [00:02<00:00, 17.97it/s]
Epoch 1 ended.
100%|██████████| 47/47 [00:02<00:00, 22.58it/s]
Epoch 2 ended.
100%|██████████| 47/47 [00:02<00:00, 15.99it/s]
Epoch 3 ended.
100%|██████████| 47/47 [00:01<00:00, 27.35it/s]
Epoch 4 ended.
100%|██████████| 47/47 [00:01<00:00, 25.40it/s]
Epoch 5 ended.
100%|██████████| 47/47 [00:01<00:00, 26.58it/s]
Epoch 6 ended.
100%|██████████| 47/47 [00:01<00:00, 26.86it/s]
Epoch 7 ended.
100%|██████████| 47/47 [00:02<00:00, 22.81it/s]
Epoch 8 ended.
100%|██████████| 47/47 [00:02<00:00, 21.06it/s]
Epoch 9 ended.
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 16/16 [00:00<00:00, 49.10it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_

In [4]:
wandb.finish()

Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000,█▁▁▁▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001,▁█▂▁▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002,▁▁█▃▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003,▁▁▁█▁
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004,▁▁▁▁█
Top1_Acc_Stream/eval_phase/test_stream/Task000,▁▂▅█▃
TrainingExperience,▁▃▅▆█
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000,0.0
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001,0.0
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002,0.00448
Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003,0.01393
